In [9]:
import pandas as pd 
import numpy as np
quantiles = np.linspace(0.05, 0.95, 19)
quantiles = [np.round(x,3) for x in quantiles]


df = pd.concat([pd.read_csv("./data/test_data.csv")]*100)
preds_df = pd.DataFrame(np.random.rand(df.shape[0], 19), columns = [f"ADJ_PRED_RENTAL_DAYS_Q_{q}" for q in quantiles])
preds_df.head()

,ADJ_PRED_RENTAL_DAYS_Q_0.05,ADJ_PRED_RENTAL_DAYS_Q_0.1,ADJ_PRED_RENTAL_DAYS_Q_0.15,ADJ_PRED_RENTAL_DAYS_Q_0.2,ADJ_PRED_RENTAL_DAYS_Q_0.25,ADJ_PRED_RENTAL_DAYS_Q_0.3,ADJ_PRED_RENTAL_DAYS_Q_0.35,ADJ_PRED_RENTAL_DAYS_Q_0.4,ADJ_PRED_RENTAL_DAYS_Q_0.45,ADJ_PRED_RENTAL_DAYS_Q_0.5,ADJ_PRED_RENTAL_DAYS_Q_0.55,ADJ_PRED_RENTAL_DAYS_Q_0.6,ADJ_PRED_RENTAL_DAYS_Q_0.65,ADJ_PRED_RENTAL_DAYS_Q_0.7,ADJ_PRED_RENTAL_DAYS_Q_0.75,ADJ_PRED_RENTAL_DAYS_Q_0.8,ADJ_PRED_RENTAL_DAYS_Q_0.85,ADJ_PRED_RENTAL_DAYS_Q_0.9,ADJ_PRED_RENTAL_DAYS_Q_0.95
0,0.612409,0.339204,0.421997,0.635637,0.715179,0.175037,0.835771,0.635742,0.996228,0.397349,0.435785,0.434696,0.278700,0.528392,0.103009,0.551088,0.562664,0.910327,0.482562
1,0.136184,0.179493,0.642604,0.802087,0.390755,0.774531,0.106287,0.457173,0.445954,0.221159,0.396974,0.068307,0.878147,0.208141,0.159729,0.771871,0.863750,0.484339,0.567970
2,0.773058,0.725020,0.985854,0.486944,0.903135,0.133791,0.638212,0.218420,0.804374,0.833218,0.901764,0.867459,0.806830,0.918599,0.491360,0.384782,0.164244,0.200164,0.285700
3,0.980917,0.671841,0.543893,0.171688,0.792406,0.165464,0.582047,0.914505,0.707720,0.142157,0.548384,0.819381,0.401135,0.009962,0.812562,0.854662,0.518797,0.531587,0.228141
4,0.329598,0.431315,0.974736,0.674747,0.415036,0.767965,0.094605,0.872784,0.623992,0.201647,0.977327,0.179874,0.703770,0.873025,0.914463,0.520332,0.052506,0.241923,0.054537


In [10]:
from pathlib import Path
import yaml
import time
import datarobot as dr
client = dr.Client() 
deployment_conf_path = Path("./deployment_conf.yaml")
if deployment_conf_path.exists():
    with open(deployment_conf_path, "r") as f:
        deployment_conf = yaml.load(f, Loader = yaml.SafeLoader)
else:
    raise Exception("Deployment configuration does not exist!! Monitoring not available")
    

In [11]:
## check number of versions and delete old ones
def purge_old_dataset_version(dataset_url):
    dataset_versions = client.get(dataset_url).json()
    if "offset" in dataset_url:
        print("off set present, deleting old versions")
        for d in dataset_versions["data"]:
            print(f"datasets/{d['datasetId']}/versions/{d['versionId']}/")
            delete_req = client.delete(f"datasets/{d['datasetId']}/versions/{d['versionId']}/")
            print(delete_req)
    if next := dataset_versions.get("next"):
        print(next)
        print(dataset_url)
        query_parameters = next.split("/")[-1]
        next_url = os.path.join( dataset_url.split("?")[0], query_parameters)
        print(next_url)
        purge_old_dataset_version(next_url)          

## this will maintain the 99 latest versions of a dataset, so plus the new version from the next cell, that will put us at 100 versions


In [12]:
new_prediction_df = df.join(preds_df)
if prediction_dataset_id := deployment_conf.get("prediction_dataset_id"):
    print("prediction dataset id present, registering new version")
    purge_old_dataset_version(f"datasets/{prediction_dataset_id}/versions?limit=99")
    prediction_dataset = dr.Dataset.create_version_from_in_memory_data(prediction_dataset_id, new_prediction_df)
else:
    print("register prediction dataset")
    prediction_dataset = dr.Dataset.create_from_in_memory_data(new_prediction_df, fname = f"Rental Calc Prediction Data")    
deployment_conf["prediction_dataset_id"] = prediction_dataset.id     

prediction dataset id present, registering new version


In [5]:
import json
import requests
for model in deployment_conf.get("deployments"):
    if model.get("batch_monitoring_job_id"):
        print("monitoring job exists")
    else: 
        print("creating monitoring job")
        monitoring_job_payload = {
            "deploymentId":model["deployment_id"],
            "monitoringAggregation": None,
            "intakeSettings":{"type":"dataset","datasetId":deployment_conf["prediction_dataset_id"]},
            "name":f"Rental Calc Monitoring {model['prediction_column']}",
            "enabled":False,
            "monitoringColumns": {"predictionsColumns":model["prediction_column"]}}
        monitoring_job_response = client.post("batchMonitoringJobDefinitions/", data = monitoring_job_payload)
        monitoring_job_response.raise_for_status()
        payload_patch = {"monitoringAggregation":None}
        batch_monitoring_job_id = monitoring_job_response.json()["id"]
        patch_response = requests.patch( f"{client.endpoint}/batchMonitoringJobDefinitions/{batch_monitoring_job_id}/", 
                      headers = { 
                          "Authorization": f"Bearer {client.token}",
                          'Content-Type': "application/json" 
                      },
                      data = json.dumps(payload_patch))
        model["batch_monitoring_job_id"] = batch_monitoring_job_id

creating monitoring job
creating monitoring job
creating monitoring job
creating monitoring job
creating monitoring job
creating monitoring job
creating monitoring job
creating monitoring job
creating monitoring job
creating monitoring job
creating monitoring job
creating monitoring job
creating monitoring job
creating monitoring job
creating monitoring job
creating monitoring job
creating monitoring job
creating monitoring job
creating monitoring job


In [6]:
with open("deployment_conf.yaml", "w") as f:
    f.write(yaml.dump(deployment_conf))

In [8]:

for model in deployment_conf["deployments"]:
    print("running monitoring job")
    job_run_payload = {"jobDefinitionId":model["batch_monitoring_job_id"]}
    job_run_response = client.post("batchJobs/fromJobDefinition/", data = job_run_payload)
    job_run_response.raise_for_status()

running monitoring job
running monitoring job
running monitoring job
running monitoring job
running monitoring job
running monitoring job
running monitoring job
running monitoring job
running monitoring job
running monitoring job
running monitoring job
running monitoring job
running monitoring job
running monitoring job
running monitoring job
running monitoring job
running monitoring job
running monitoring job
running monitoring job
